# ABS Quarterly Financial Accounts 5232

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_meta_constants,
    get_plot_constants,
    get_ABS_meta_and_data,
    get_identifier,
)
from plotting import (
    line_plot,
    set_chart_dir,
    recalibrate_series,
    clear_chart_dir,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999

## Get data from ABS

In [2]:
catalogue_id = "5232"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_dict = get_ABS_meta_and_data(catalogue_id)
if abs_dict is None:
    sys.exit(-1)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
We need to download this file from the ABS ...
Saving ABS download to cache.
Extracting DataFrames from the zip-file ...


## Plot

### Set-up

In [3]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Household balance sheet

In [4]:
table = "35"
data = abs_dict[table]
series_type = "Original"
plots = meta[meta["Table"] == table][did_col]

for plot in plots:
    ident, orig_units = get_identifier(meta, plot, series_type, table)

    # manage long wordy chart titles
    MAX_LEN = 50
    title = f"HBS: {plot[:-2]}"
    footnote = ""
    replaceable = {
        "- Non-financial -": ["- NF -", " NF = Non-financial."],
        "- Fixed assets -": ["- FA -", " FA = Fixed assets."],
        "- Financial assets -": ["- FA -", " FA = Financial assets."],
    }
    for foundling, [replacement, fnote] in replaceable.items():
        if foundling in title and len(title) > MAX_LEN:
            title = title.replace(foundling, replacement)
            footnote = f"{footnote}{fnote}"
    if len(title) > MAX_LEN:
        splits = title.split(" - ")
        title = " - ".join(splits[:-1]) + " -\n" + splits[-1]

    series = data[ident]
    series, units = recalibrate_series(series, orig_units)

    # plot
    line_plot(
        series,
        starts=plot_times,
        tags=plot_tags,
        title=title,
        ylabel=f"{units} current prices",
        rfooter=f"{source} {table}",
        lfooter=f"{series_type.capitalize()} series."
        f" HBS = Household balance sheet.{footnote}",
        # show=True,
    )

In [5]:
# Land and dwelling assets - side by side
table = "35"
data = abs_dict[table]
series_type = "Original"
land = "Non-financial - Non-produced assets - Land ;"
dwellings = "Non-financial - Produced - Fixed assets - Dwellings ;"
combined = {}
for name, series in zip(("Land", "Dwellings"), (land, dwellings)):
    ident, units = get_identifier(meta, series, series_type, table)
    combined[name] = data[ident]
combined = pd.DataFrame(combined)
title = "Household Balance Sheet for Land and Dwellings"
line_plot(
    combined,
    title=title,
    ylabel=f"{units} current prices",
    rfooter=f"{source} {table}",
    lfooter=f"{series_type.capitalize()} series",
    # show=True,
)

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue May 23 2023 08:08:31

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.13.2

pandas    : 2.0.1
matplotlib: 3.7.1
sys       : 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]

Watermark: 2.3.1



In [7]:
print("Finished")

Finished
